In [ ]:
# Generated by Snowflake Copilot
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from snowflake.snowpark.context import get_active_session
import warnings
warnings.filterwarnings('ignore')


@st.cache_data
def load_and_prepare_data():
    session = get_active_session()
    table_name = "DETECCION_FRAUDE.FRAUD_DETECTION_SCHEMA.TRANSACCIONES_AVANZADAS"
    df = session.table(table_name).to_pandas()
    
    feature_columns = [col for col in df.columns if col not in ['TRANSACTION_ID', 'IS_FRAUD', 'CREATED_AT', 'UPDATED_AT']]
    X = df[feature_columns].copy()
    y = df['IS_FRAUD'].copy()
    
    categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
    label_encoders = {}
    for col in categorical_columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        label_encoders[col] = le
    
    return X, y, label_encoders, feature_columns

@st.cache_resource
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    return model, X_train

st.title("🔍 Detector de Fraude Interactivo")
st.markdown("### Ajusta las variables y predice la probabilidad de fraude")

try:
    X, y, label_encoders, feature_columns = load_and_prepare_data()
    model, X_train = train_model(X, y)
    
    st.success(f"✅ Modelo entrenado exitosamente con {len(feature_columns)} características")
    
    col1, col2 = st.columns(2)
    
    with col1:
        st.subheader("💰 Variables de Transacción")
        monto_transaccion = st.slider("Monto de Transacción ($)", 0, 50000, 1000, 100)
        distancia_domicilio = st.slider("Distancia del Domicilio (km)", 0, 10000, 100, 50)
        diferencia_residencia = st.slider("Diferencia Residencia-Compra (km)", 0, 10000, 50, 50)
        variacion_gasto = st.slider("Variación Gasto Mes Anterior ($)", -5000, 5000, 0, 100)
        hora_dia = st.slider("Hora del Día", 0, 23, 12, 1)
        
        st.subheader("🌍 Variables Geográficas")
        pais_origen = st.selectbox("País Origen", ["USA", "MEX", "CAN", "ESP", "FRA", "GBR"])
        pais_destino = st.selectbox("País Destino", ["USA", "MEX", "CAN", "ESP", "FRA", "GBR", "RUS", "CHN"])
        diferencia_horaria = st.slider("Diferencia Horaria (horas)", 0, 24, 0, 1)
        
    with col2:
        st.subheader("🔒 Variables de Seguridad")
        usa_vpn = st.checkbox("Usa VPN/Proxy")
        usa_autocompletado = st.checkbox("Usa Autocompletado/Copy-Paste")
        usa_emails_temporales = st.checkbox("Usa Emails Temporales")
        dispositivo_confiable = st.checkbox("Dispositivo Confiable")
        cambio_geolocalizacion_imposible = st.checkbox("Cambio Geolocalización Imposible")
        
        st.subheader("👤 Variables de Comportamiento")
        antiguedad_cuenta = st.slider("Antigüedad Cuenta (días)", 1, 3650, 365, 30)
        intentos_login_24h = st.slider("Intentos Login 24h", 1, 20, 3, 1)
        velocidad_digitacion = st.slider("Velocidad Digitación (CPM)", 10, 200, 60, 5)
        transacciones_internacionales = st.slider("Transacciones Internacionales", 0, 10, 0, 1)
        
        st.subheader("💳 Variables Financieras")
        porcentaje_uso_limite = st.slider("% Uso Límite Crédito", 0, 100, 30, 5)
        historial_contracargos = st.slider("Historial Contracargos", 0, 10, 0, 1)
    
    st.markdown("---")
    
    if st.button("🎯 **PREDECIR FRAUDE**", type="primary", use_container_width=True):
        user_input = {
            'MONTO_TRANSACCION': float(monto_transaccion),
            'DISTANCIA_DOMICILIO_KM': float(distancia_domicilio),
            'DIFERENCIA_RESIDENCIA_COMPRA_KM': float(diferencia_residencia),
            'VARIACION_GASTO_MES_ANTERIOR': float(variacion_gasto),
            'HORA_DIA': int(hora_dia),
            'PAIS_ORIGEN': pais_origen,
            'PAIS_DESTINO': pais_destino,
            'DIFERENCIA_HORARIA_HORAS': int(diferencia_horaria),
            'USA_VPN_PROXY': usa_vpn,
            'USO_AUTOCOMPLETADO_COPYPASTE': usa_autocompletado,
            'USA_EMAILS_TEMPORALES': usa_emails_temporales,
            'DISPOSITIVO_CONFIABLE': dispositivo_confiable,
            'CAMBIO_GEOLOCALIZACION_IMPOSIBLE': cambio_geolocalizacion_imposible,
            'ANTIGUEDAD_CUENTA_DIAS': int(antiguedad_cuenta),
            'INTENTOS_LOGIN_24H': int(intentos_login_24h),
            'VELOCIDAD_DIGITACION_CPM': float(velocidad_digitacion),
            'TRANSACCIONES_INTERNACIONALES': int(transacciones_internacionales),
            'PORCENTAJE_USO_LIMITE': float(porcentaje_uso_limite),
            'HISTORIAL_CONTRACARGOS': int(historial_contracargos)
        }
        
        input_df = pd.DataFrame([user_input])
        
        for col in feature_columns:
            if col not in input_df.columns:
                if col in X.columns:
                    input_df[col] = X[col].median()
                else:
                    input_df[col] = 0
        
        categorical_columns = ['PAIS_ORIGEN', 'PAIS_DESTINO']
        for col in categorical_columns:
            if col in input_df.columns and col in label_encoders:
                try:
                    input_df[col] = label_encoders[col].transform(input_df[col].astype(str))
                except ValueError:
                    input_df[col] = 0
        
        input_df = input_df[feature_columns]
        
        fraud_probability = model.predict_proba(input_df)[0, 1]
        fraud_prediction = model.predict(input_df)[0]
        
        st.markdown("---")
        st.subheader("📊 Resultados de la Predicción")
        
        col1, col2, col3 = st.columns(3)
        
        with col1:
            st.metric("Probabilidad de Fraude", f"{fraud_probability:.1%}")
        
        with col2:
            prediction_text = "🚨 FRAUDE" if fraud_prediction else "✅ LEGÍTIMA"
            st.metric("Predicción", prediction_text)
        
        with col3:
            if fraud_probability > 0.7:
                risk_level = "🔴 ALTO RIESGO"
            elif fraud_probability > 0.4:
                risk_level = "🟡 RIESGO MEDIO"
            else:
                risk_level = "🟢 BAJO RIESGO"
            st.metric("Nivel de Riesgo", risk_level)
        
        progress_bar = st.progress(fraud_probability)
        
        if fraud_probability > 0.6:
            st.error(f"⚠️ **ALERTA DE FRAUDE**: Probabilidad alta ({fraud_probability:.1%})")
            st.markdown("**Recomendación**: Revisar manualmente la transacción")
        elif fraud_probability > 0.3:
            st.warning(f"⚠️ **PRECAUCIÓN**: Probabilidad media ({fraud_probability:.1%})")
            st.markdown("**Recomendación**: Verificación adicional requerida")
        else:
            st.success(f"✅ **TRANSACCIÓN SEGURA**: Probabilidad baja ({fraud_probability:.1%})")
            st.markdown("**Recomendación**: Proceder con la transacción")
        
        risk_factors = []
        if usa_vpn:
            risk_factors.append("Uso de VPN/Proxy")
        if usa_autocompletado:
            risk_factors.append("Uso de autocompletado")
        if usa_emails_temporales:
            risk_factors.append("Emails temporales")
        if monto_transaccion > 5000:
            risk_factors.append(f"Monto alto (${monto_transaccion:,})")
        if distancia_domicilio > 1000:
            risk_factors.append(f"Distancia alta ({distancia_domicilio} km)")
        if cambio_geolocalizacion_imposible:
            risk_factors.append("Cambio geolocalización imposible")
        
        if risk_factors:
            st.subheader("⚠️ Factores de Riesgo Identificados")
            for factor in risk_factors:
                st.write(f"• {factor}")

except Exception as e:
    st.error(f"Error al cargar el modelo: {str(e)}")
    st.info("Asegúrate de que la tabla de datos esté disponible en Snowflake")

st.markdown("---")
st.markdown("**Desarrollado con Snowflake ML** | Modelo: Random Forest")